<a href="https://colab.research.google.com/github/Xeena2812/onallo-labor/blob/original-inceptionv4/notebook65ef06b5ad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the CIFAR-100 dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

169001437/169001437 [==============================] - 11s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [2]:
print(tf.config.list_physical_devices('GPU'))
!nvidia-smi

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Thu Apr 27 14:33:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                     

In [3]:
from keras.applications.inception_v3 import preprocess_input
from tensorflow.data import Dataset
from keras.layers import Resizing, Rescaling
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

print(x_train.shape)
print(x_test.shape)

#x_train = np.moveaxis(x_train, -1, 1)
#x_test = np.moveaxis(x_test, -1, 1)

print(x_train.shape)
print(x_test.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 32, 32, 3)
(10000, 32, 32, 3)


## Inception v4 Implementation

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from keras.metrics import TopKCategoricalAccuracy

resize_rescale = tf.keras.Sequential([
    Input(shape=(32, 32, 3)),
    Resizing(299, 299),
    Rescaling(1./255, offset=-1),
])
# create the base pre-trained model
base_model = InceptionV3(input_tensor=resize_rescale.output, weights='imagenet', include_top=False)


# add own top to pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(100, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first train only the top layers
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])




87910968/87910968 [==============================] - 3s 0us/step


In [5]:
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure


def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(x_test)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(y_test, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=np.arange(100))
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("epoch_confusion_matrix", cm_image, step=epoch)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

In [6]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])
# run previous block to train model again, but with more epochs

0 input_1
1 resizing
2 rescaling
3 conv2d
4 batch_normalization
5 activation
6 conv2d_1
7 batch_normalization_1
8 activation_1
9 conv2d_2
10 batch_normalization_2
11 activation_2
12 max_pooling2d
13 conv2d_3
14 batch_normalization_3
15 activation_3
16 conv2d_4
17 batch_normalization_4
18 activation_4
19 max_pooling2d_1
20 conv2d_8
21 batch_normalization_8
22 activation_8
23 conv2d_6
24 conv2d_9
25 batch_normalization_6
26 batch_normalization_9
27 activation_6
28 activation_9
29 average_pooling2d
30 conv2d_5
31 conv2d_7
32 conv2d_10
33 conv2d_11
34 batch_normalization_5
35 batch_normalization_7
36 batch_normalization_10
37 batch_normalization_11
38 activation_5
39 activation_7
40 activation_10
41 activation_11
42 mixed0
43 conv2d_15
44 batch_normalization_15
45 activation_15
46 conv2d_13
47 conv2d_16
48 batch_normalization_13
49 batch_normalization_16
50 activation_13
51 activation_16
52 average_pooling2d_1
53 conv2d_12
54 conv2d_14
55 conv2d_17
56 conv2d_18
57 batch_normalization_12
58

In [8]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, TerminateOnNaN
import datetime

logdir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.
model.fit(x_train, y_train,
          epochs=200,
          batch_size=512,
          validation_batch_size=512,
          callbacks=[
              TensorBoard(
                  log_dir=logdir,
                  histogram_freq=1
              ),
              EarlyStopping(
                  monitor='val_top_1_accuracy',
                  patience=10,
                  restore_best_weights=True,
              ),
              ModelCheckpoint(
                  logdir+'/checkpoint',
                  monitor='val_top_1_accuracy',
                  save_best_only=True,
              ),
              TerminateOnNaN(),
              cm_callback,
          ],
)


Epoch 1/200
98/98 [==============================] - ETA: 0s - loss: 1.4304 - top_1_accuracy: 0.6113 - top_5_accuracy: 0.8676

313/313 [==============================] - 44s 128ms/step


NameError: ignored

In [ ]:
%load_external tensorboard

In [ ]:
%tensorboard --logdir ./logs/fit